<a href="https://colab.research.google.com/github/YeweiYuan/melody/blob/master/gpu_melody.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import os
from music21 import converter, instrument, note, chord, stream
import numpy as np

In [0]:
#读取训练数据的Note
filepath='../home/all/'
files=os.listdir(filepath)
Notes=[]
for file in files:
  try:
        stream = converter.parse(filepath+file)
        instru = instrument.partitionByInstrument(stream)
        if instru:  # 如果有乐器部分，取第一个乐器部分
            notes = instru.parts[0].recurse()
        else:  #如果没有乐器部分，直接取note
            notes = stream.flat.notes
        for element in notes:
              # 如果是 Note 类型，取音调
              # 如果是 Chord 类型，取音调的序号,存int类型比较容易处理
            if isinstance(element, note.Note):
                Notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                Notes.append('.'.join(str(n) for n in element.normalOrder))
  except:
      pass
  #记得注释掉
  #with open('Note', 'a+')as f:
      #f.write(str(Notes))

In [0]:
notes = Notes

notes_len=len(set(notes))
note_name=sorted(set(i for i in notes))#获得排序的不重复的音符名字
sequence_length = 100 #序列长度
note_dict=dict((j,i) for i,j in enumerate(note_name))#设计一个字典，把音符转换成数字，方便训练

network_input = []#创建输入序列
network_output = []#创建输出序列
for i in range(0, len(notes) - sequence_length):
    #输入100个，输出1个
    sequence_in = notes[i: i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_dict[k] for k in sequence_in])
    network_output.append(note_dict[sequence_out])

In [0]:
network_input = np.reshape(network_input, (len(network_input), sequence_length, 1))
normal_network_input = network_input / float(notes_len) #归一化

network_output = tf.keras.utils.to_categorical(network_output)#输出布尔矩阵，配合categorical_crossentropy 算法使用

In [0]:
a = normal_network_input[0:2]
b = network_output[0:2]

In [0]:
def get_model(inputs, notes_len, weights_file=None):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(512,input_shape=(inputs.shape[1], inputs.shape[2]),return_sequences=True))#512层神经元，return_sequences=True表示返回所有的输出序列
    model.add(tf.keras.layers.Dropout(0.3))  # 丢弃 30% 神经元，防止过拟合
    model.add(tf.keras.layers.LSTM(512, return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.LSTM(512))  # return_sequences 是默认的 False，只返回输出序列的最后一个
    model.add(tf.keras.layers.Dense(256))  # 256 个神经元的全连接层
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(notes_len))  # 输出的数目等于所有不重复的音调的数目
    model.add(tf.keras.layers.Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adadelta')
 
    if weights_file is not None:
        model.load_weights(weights_file)
 
    return model

In [0]:
np.set_printoptions(threshold=6)

In [9]:
normal_network_input[1]

array([[0.978907  ],
       [0.978907  ],
       [0.978907  ],
       [0.57718121],
       [0.77660594],
       [0.978907  ],
       [0.95589645],
       [0.978907  ],
       [0.5033557 ],
       [0.978907  ],
       [0.978907  ],
       [0.93959732],
       [0.57718121],
       [0.12655801],
       [0.5033557 ],
       [0.978907  ],
       [0.978907  ],
       [0.978907  ],
       [0.57718121],
       [0.93959732],
       [0.978907  ],
       [0.95589645],
       [0.978907  ],
       [0.978907  ],
       [0.5033557 ],
       [0.978907  ],
       [0.978907  ],
       [0.93959732],
       [0.57718121],
       [0.12655801],
       [0.5033557 ],
       [0.978907  ],
       [0.978907  ],
       [0.978907  ],
       [0.57718121],
       [0.58485139],
       [0.93959732],
       [0.978907  ],
       [0.95589645],
       [0.978907  ],
       [0.978907  ],
       [0.5033557 ],
       [0.978907  ],
       [0.978907  ],
       [0.93959732],
       [0.57718121],
       [0.12655801],
       [0.503

In [0]:
model = get_model(normal_network_input,notes_len)

In [11]:
filepath = "weights-{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='loss',  # 监控的对象是loss
    verbose=0,
    save_best_only=True,
    mode='min'  # 如果监控对象是val_acc则取max，是loss则取min
)
callbacks_list = [checkpoint]
model.fit(normal_network_input, network_output, epochs=100, batch_size=128, callbacks=callbacks_list) #整体迭代100次，每小批128个

Epoch 1/100
442/442 [==============================] - 59s 133ms/step - loss: 6.9474
Epoch 2/100
442/442 [==============================] - 62s 141ms/step - loss: 6.9446
Epoch 3/100
442/442 [==============================] - 62s 140ms/step - loss: 6.9417
Epoch 4/100
442/442 [==============================] - 62s 140ms/step - loss: 6.9384
Epoch 5/100
442/442 [==============================] - 62s 140ms/step - loss: 6.9347
Epoch 6/100
442/442 [==============================] - 62s 140ms/step - loss: 6.9302
Epoch 7/100
442/442 [==============================] - 62s 140ms/step - loss: 6.9244
Epoch 8/100
442/442 [==============================] - 62s 140ms/step - loss: 6.9168
Epoch 9/100
442/442 [==============================] - 62s 140ms/step - loss: 6.9056
Epoch 10/100
442/442 [==============================] - 62s 140ms/step - loss: 6.8877
Epoch 11/100
442/442 [==============================] - 62s 140ms/step - loss: 6.8534
Epoch 12/100
442/442 [==============================] - 62s 140

In [12]:
model.predict(normal_network_input)

array([[0.01083246, 0.00336389, 0.00095293, ..., 0.00486094, 0.00395192,
        0.00083549],
       [0.01082871, 0.00336434, 0.00095315, ..., 0.00486103, 0.00395107,
        0.00083582],
       [0.01082592, 0.00336442, 0.00095331, ..., 0.00486103, 0.00395048,
        0.00083603],
       ...,
       [0.01086014, 0.00335825, 0.00095063, ..., 0.00486154, 0.00395987,
        0.00083233],
       [0.01086102, 0.00335798, 0.00095053, ..., 0.00486158, 0.00396016,
        0.00083221],
       [0.01086243, 0.00335755, 0.0009504 , ..., 0.00486158, 0.0039606 ,
        0.00083203]], dtype=float32)

In [13]:
model.evaluate(a,b)

1/1 [==============================] - 0s 1ms/step - loss: 3.4243


3.424316883087158

In [14]:
randindex = np.random.randint(0, len(network_input) - 1)
notedic = dict((i,j) for i, j in enumerate(note_name))    # 把刚才的整数还原成音调
pattern = list(network_input[randindex])
predictions = []
#随机生成1000个音符
for note_index in range(500):
    #pattern = list(network_input[np.random.randint(0,500)])
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    
    prediction_input = prediction_input / float(notes_len)#归一化
    prediction = model.predict(prediction_input)

    if note_index <5:
      print(prediction)
    index = np.argmax(prediction)
    if note_index <5:
      print(index)
    result = notedic[index]
    predictions.append(result)
    # 往后移动
    pattern.append(np.array([index]))
    pattern = pattern[1:len(pattern)]

[[0.01067295 0.00337662 0.00096615 ... 0.00484987 0.00391396 0.00085095]]
1021
[[0.01067835 0.00337568 0.00096571 ... 0.00484999 0.00391549 0.00085037]]
1021
[[0.01068478 0.00337451 0.00096519 ... 0.00485017 0.00391727 0.00084968]]
1021
[[0.01069233 0.00337316 0.00096458 ... 0.00485036 0.00391932 0.00084887]]
1021
[[0.010701   0.00337167 0.00096389 ... 0.0048506  0.00392164 0.00084795]]
1021


['F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',
 'F#2',


In [0]:
from music21 import stream as st
offset = 0
output_notes = []
# 生成 Note（音符）或 Chord（和弦）对象
for data in predictions:
    if ('.' in data) or data.isdigit():
        notes_in_chord = data.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(data)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 1
# 创建音乐流（Stream）
midi_stream = st.Stream(output_notes)
# 写入 MIDI 文件
midi_stream.write('midi', fp='output1.mid')

'output1.mid'